<a href="https://colab.research.google.com/github/donghyundavidchoi/Data_Analysis/blob/main/%EB%AA%A8%EB%8D%B8%EB%A7%81/%EB%8B%B9%EB%87%A8%EB%B3%91%EC%98%88%EC%B8%A1%EB%AA%A8%EB%8D%B8_index_drop_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 당뇨병 예측 모델
# 값이 0인 변수에 대한 결측치 처리

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-/view?usp=sharing
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Downloading...
From: https://drive.google.com/uc?id=1txRVXp-Eh222PzIeTQ-fuAoUYFR9f7q-
To: /content/kaggle.json
100% 73.0/73.0 [00:00<00:00, 93.8kB/s]


In [2]:
!kaggle datasets download -d uciml/pima-indians-diabetes-database

  0% 0.00/8.91k [00:00<?, ?B/s]
100% 8.91k/8.91k [00:00<00:00, 8.11MB/s]


In [3]:
!unzip /content/pima-indians-diabetes-database.zip

Archive:  /content/pima-indians-diabetes-database.zip
  inflating: diabetes.csv            


In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("/content/diabetes.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Outcome')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 9), (154, 9), (614, 2), (154, 2))

-------

# EDA

In [53]:
X_train_id = X_train['id']
X_train = X_train.drop(columns='id', axis=1)
X_test_id = X_test['id']
X_test = X_test.drop(columns='id', axis=1)

In [54]:
X_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
147,2,106,64,35,119,30.5,1.400,34
344,8,95,72,0,0,36.8,0.485,57
390,1,100,66,29,196,32.0,0.444,42
150,1,136,74,50,204,37.4,0.399,24
132,3,170,64,37,225,34.5,0.356,30
...,...,...,...,...,...,...,...,...
621,2,92,76,20,0,24.2,1.698,28
128,1,117,88,24,145,34.5,0.403,40
57,0,100,88,60,110,46.8,0.962,31
341,1,95,74,21,73,25.9,0.673,36


In [55]:
X_train.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.845277,120.705212,68.530945,20.982085,82.229642,31.948534,0.477471,33.319218
std,3.388952,32.047977,19.761915,15.965031,116.525772,7.734385,0.339811,11.940953
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.400000,0.244250,24.000000
50%,3.000000,117.000000,72.000000,23.000000,44.000000,32.250000,0.377000,29.000000
75%,6.000000,140.000000,80.000000,33.000000,130.000000,36.600000,0.628500,41.000000
max,17.000000,199.000000,114.000000,99.000000,846.000000,59.400000,2.420000,81.000000


In [56]:
X_train.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Pregnancies,1.000000,0.157231,0.151080,-0.086042,-0.079270,0.020483,-0.040809,0.530117
Glucose,0.157231,1.000000,0.160031,0.058158,0.313152,0.243888,0.145261,0.265131
BloodPressure,0.151080,0.160031,1.000000,0.233607,0.115477,0.284482,0.049321,0.234298
SkinThickness,-0.086042,0.058158,0.233607,1.000000,0.431224,0.406174,0.182880,-0.137826
Insulin,-0.079270,0.313152,0.115477,0.431224,1.000000,0.214843,0.189927,-0.064915
BMI,0.020483,0.243888,0.284482,0.406174,0.214843,1.000000,0.141877,0.061852
DiabetesPedigreeFunction,-0.040809,0.145261,0.049321,0.182880,0.189927,0.141877,1.000000,0.041031
Age,0.530117,0.265131,0.234298,-0.137826,-0.064915,0.061852,0.041031,1.000000


In [57]:
X_test

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
258,1,193,50,16,375,25.9,0.655,24
220,0,177,60,29,478,34.6,1.072,21
438,1,97,70,15,0,18.2,0.147,21
130,4,173,70,14,168,29.7,0.361,33
730,3,130,78,23,79,28.4,0.323,34
...,...,...,...,...,...,...,...,...
678,3,121,52,0,0,36.0,0.127,25
500,2,117,90,19,71,25.2,0.313,21
256,3,111,56,39,0,30.1,0.557,30
80,3,113,44,13,0,22.4,0.140,22


In [58]:
X_test.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000
mean,3.844156,121.649351,71.396104,18.759740,70.110390,32.168182,0.449571,32.928571
std,3.302069,31.762915,17.516908,15.827937,109.815982,8.479377,0.295059,11.041873
min,0.000000,44.000000,0.000000,0.000000,0.000000,0.000000,0.088000,21.000000
25%,1.000000,99.250000,64.000000,0.000000,0.000000,27.100000,0.240750,24.000000
50%,3.000000,117.500000,72.000000,19.500000,0.000000,31.950000,0.363500,29.000000
75%,6.000000,141.500000,80.000000,31.000000,109.000000,36.275000,0.600750,39.000000
max,13.000000,197.000000,122.000000,56.000000,543.000000,67.100000,1.893000,70.000000


In [59]:
X_test.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Pregnancies,1.000000,0.014307,0.097799,-0.064249,-0.048924,0.007292,0.000662,0.608454
Glucose,0.014307,1.000000,0.117076,0.057775,0.414107,0.137341,0.104199,0.258032
BloodPressure,0.097799,0.117076,1.000000,0.111142,-0.022342,0.275138,0.012380,0.272541
SkinThickness,-0.064249,0.057775,0.111142,1.000000,0.454267,0.349918,0.181670,-0.014946
Insulin,-0.048924,0.414107,-0.022342,0.454267,1.000000,0.137020,0.155004,0.059061
BMI,0.007292,0.137341,0.275138,0.349918,0.137020,1.000000,0.141003,-0.063787
DiabetesPedigreeFunction,0.000662,0.104199,0.012380,0.181670,0.155004,0.141003,1.000000,-0.006216
Age,0.608454,0.258032,0.272541,-0.014946,0.059061,-0.063787,-0.006216,1.000000


In [67]:
y_train

,id,Outcome
147,147,0
344,344,0
390,390,0
150,150,0
132,132,1
...,...,...
621,621,0
128,128,1
57,57,0
341,341,0


In [60]:
print('Number of 0 in X_train Glucose:', len(X_train['Glucose'][X_train['Glucose'] == 0]))
print('Number of 0 in X_train BloodPressure:', len(X_train['BloodPressure'][X_train['BloodPressure'] == 0]))
print('Number of 0 in X_train SkinThickness:', len(X_train['SkinThickness'][X_train['SkinThickness'] == 0]))
print('Number of 0 in X_train Insulin:', len(X_train['Insulin'][X_train['Insulin'] == 0]))
print('Number of 0 in X_train BMI:', len(X_train['BMI'][X_train['BMI'] == 0]))
print()
print('Number of 0 in X_test Glucose:', len(X_test['Glucose'][X_test['Glucose'] == 0]))
print('Number of 0 in X_test BloodPressure:', len(X_test['BloodPressure'][X_test['BloodPressure'] == 0]))
print('Number of 0 in X_test SkinThickness:', len(X_test['SkinThickness'][X_test['SkinThickness'] == 0]))
print('Number of 0 in X_test Insulin:', len(X_test['Insulin'][X_test['Insulin'] == 0]))
print('Number of 0 in X_test BMI:', len(X_test['BMI'][X_test['BMI'] == 0]))

Number of 0 in X_train Glucose: 5
Number of 0 in X_train BloodPressure: 31
Number of 0 in X_train SkinThickness: 175
Number of 0 in X_train Insulin: 287
Number of 0 in X_train BMI: 9

Number of 0 in X_test Glucose: 0
Number of 0 in X_test BloodPressure: 4
Number of 0 in X_test SkinThickness: 52
Number of 0 in X_test Insulin: 87
Number of 0 in X_test BMI: 2


-------

# Glucose의 경우 train 데이터에서만 결측이 확인됨으로 결측치가 포함된 행을 삭제하는 방식으로 진행

In [61]:
glucose_0_index = X_train['Glucose'][X_train['Glucose'] == 0].index
glucose_0_index

Int64Index([182, 75, 342, 502, 349], dtype='int64')

In [62]:
X_train = X_train.drop(index=glucose_0_index, axis=1)
y_train = y_train.drop(index=glucose_0_index, axis=1)
print(X_train.shape)
print(y_train.shape)

(609, 8)
(609, 2)


----------

# BloodPressure, SkinThickness, Insulin, BMI의 셜측치를 평균값으로 채우기

In [63]:
cols = ['BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in cols:
    X_train[col] = X_train[col].replace(0, X_train[col].mean())

In [64]:
print('Number of 0 in X_train Glucose:', len(X_train['Glucose'][X_train['Glucose'] == 0]))
print('Number of 0 in X_train BloodPressure:', len(X_train['BloodPressure'][X_train['BloodPressure'] == 0]))
print('Number of 0 in X_train SkinThickness:', len(X_train['SkinThickness'][X_train['SkinThickness'] == 0]))
print('Number of 0 in X_train Insulin:', len(X_train['Insulin'][X_train['Insulin'] == 0]))
print('Number of 0 in X_train BMI:', len(X_train['BMI'][X_train['BMI'] == 0]))

Number of 0 in X_train Glucose: 0
Number of 0 in X_train BloodPressure: 0
Number of 0 in X_train SkinThickness: 0
Number of 0 in X_train Insulin: 0
Number of 0 in X_train BMI: 0


In [98]:
cols = ['BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in cols:
    X_test[col] = X_test[col].replace(0, X_test[col].mean())

In [99]:
print('Number of 0 in X_test Glucose:', len(X_test['Glucose'][X_test['Glucose'] == 0]))
print('Number of 0 in X_test BloodPressure:', len(X_test['BloodPressure'][X_test['BloodPressure'] == 0]))
print('Number of 0 in X_test SkinThickness:', len(X_test['SkinThickness'][X_test['SkinThickness'] == 0]))
print('Number of 0 in X_test Insulin:', len(X_test['Insulin'][X_test['Insulin'] == 0]))
print('Number of 0 in X_test BMI:', len(X_test['BMI'][X_test['BMI'] == 0]))

Number of 0 in X_test Glucose: 0
Number of 0 in X_test BloodPressure: 0
Number of 0 in X_test SkinThickness: 0
Number of 0 in X_test Insulin: 0
Number of 0 in X_test BMI: 0


--------

# 모델링 : SVC

In [100]:
from sklearn.svm import SVC

In [101]:
model_SVC = SVC()
model_SVC.fit(X_train, y_train['Outcome'])
print(round(model_SVC.score(X_train, y_train['Outcome']), 2))
print(round(model_SVC.score(X_test, y_test['Outcome']), 2))

0.76
0.76


--------

# 모델링 : LogisticRegression

In [102]:
from sklearn.linear_model import LogisticRegression

In [103]:
model_LR = LogisticRegression()
model_LR.fit(X_train, y_train['Outcome'])
print(round(model_LR.score(X_train, y_train['Outcome']), 2))
print(round(model_LR.score(X_test, y_test['Outcome']), 2))

0.77
0.72


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


----------

# SVC 모델의 예상값 저장

In [106]:
prediction = model_SVC.predict(X_test)

In [107]:
prediction

array([1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0])

In [108]:
output = pd.DataFrame({'id' : X_test_id, 'Outcome' : prediction})

In [109]:
output

,id,Outcome
258,258,1
220,220,1
438,438,0
130,130,1
730,730,0
...,...,...
678,678,0
500,500,0
256,256,0
80,80,0


In [110]:
output.to_csv('20220601.csv', index=False)